In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim

In [39]:
# تعریف ترانسفورم برای پیش‌پردازش تصاویر
transform = transforms.Compose([
    transforms.ToTensor(),  # تبدیل تصویر به تنسور پایتورچ
])

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 120)  # لایه پنهان اول
        self.fc2 = nn.Linear(120, 84)       # لایه پنهان دوم
        self.fc3 = nn.Linear(84, 10)        # لایه خروجی
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # تغییر ابعاد برای ورودی به شبکه
        x = F.relu(self.fc1(x))  # فعال‌سازی ReLU برای لایه پنهان اول
        x = F.relu(self.fc2(x))  # فعال‌سازی ReLU برای لایه پنهان دوم
        x = self.fc3(x)          # لایه خروجی بدون فعال‌سازی آخری
        return x

# ایجاد نمونه از مدل
model = MLP()
print(model) 

MLP(
  (fc1): Linear(in_features=784, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [40]:
# محاسبه تعداد به‌روزرسانی‌ها (تکرارها)
total_samples = 12000
batch_size = 200
num_epochs = 5

total_updates = (total_samples // batch_size) * num_epochs
print("تعداد به‌روزرسانی‌ها (تکرارها):", total_updates)

تعداد به‌روزرسانی‌ها (تکرارها): 300


In [41]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# ایجاد نمونه از مدل
model = MLP()

# تعریف تابع خطا و بهینه‌ساز
criterion = nn.CrossEntropyLoss()

# حساب کردن هزینه برای یک دسته از داده‌ها
for images, labels in train_loader:
    outputs = model(images)
    loss = criterion(outputs, labels)
    print("هزینه برای یک دسته از داده‌ها:", loss.item())
    break  # برای یک دسته از داده‌ها از حلقه خارج می‌شویم

هزینه برای یک دسته از داده‌ها: 2.3149290084838867


In [42]:
# تعریف بهینه‌ساز (استفاده از SGD)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# اجرای Backpropagation و به‌روزرسانی وزن‌ها
for images, labels in train_loader:
    # پاک کردن گرادیان‌ها
    optimizer.zero_grad()
    
    # محاسبه خروجی‌ها
    outputs = model(images)
    # محاسبه خطای شبکه
    loss = criterion(outputs, labels)
    # محاسبه گرادیان‌ها
    loss.backward()
    
    # به‌روزرسانی وزن‌ها
    optimizer.step()

In [43]:
# تابع محاسبه دقت
def calculate_accuracy(model, data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total


In [45]:
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
# آموزش مدل
for epoch in range(num_epochs):
    # آموزش مدل
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # ارزیابی دقت و هزینه آزمون برای این ایپاک
    model.eval()
    test_loss = 0
    test_accuracy = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            test_accuracy += (predicted == labels).sum().item()
    test_loss /= len(test_loader)
    test_accuracy /= len(test_dataset)
    
    # گزارش دقت و هزینه آزمون برای این ایپاک
    print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/5, Test Loss: 0.2647, Test Accuracy: 0.9255
Epoch 2/5, Test Loss: 0.2502, Test Accuracy: 0.9300
Epoch 3/5, Test Loss: 0.2387, Test Accuracy: 0.9332
Epoch 4/5, Test Loss: 0.2259, Test Accuracy: 0.9345
Epoch 5/5, Test Loss: 0.2187, Test Accuracy: 0.9354


In [51]:
import pandas as pd

# ذخیره هزینه و دقت برای هر ایپاک
epoch_results = []

# آموزش مدل
for epoch in range(num_epochs):
    # آموزش مدل
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # ارزیابی دقت و هزینه آزمون برای این ایپاک
    model.eval()
    test_loss = 0
    test_accuracy = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            test_accuracy += (predicted == labels).sum().item()
    test_loss /= len(test_loader)
    test_accuracy /= len(test_dataset)
    
    # ذخیره نتایج ایپاک جاری
    epoch_results.append({'Epoch': epoch+1, 'Test Loss': test_loss, 'Test Accuracy': test_accuracy})

# تبدیل نتایج به دیتافریم پانداس
results_df = pd.DataFrame(epoch_results)

# چاپ دیتافریم به صورت جدول
results_df

,Epoch,Test Loss,Test Accuracy
0,1,0.105785,0.9697
1,2,0.103919,0.9696
2,3,0.101571,0.9697
3,4,0.103670,0.9688
4,5,0.098866,0.9700
